In [1]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.end_of_field = False
        self.description = ""

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, field, description):
        current = self.root
        for char in field:
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]
        current.end_of_field = True
        current.description = description
    
    def search(self, field):
        current = self.root
        for char in field:
            if char not in current.children:
                return None
            current = current.children[char]
        if current.end_of_word:
            return current.description
        return None
    
    def display_tree(self):
    
    def search_knn(self, field, k_neighbors = 1):

In [2]:
trie = Trie()

trie.insert("banana", "A long, curved fruit with a yellow skin and soft, sweet flesh.")
trie.insert("cherry", "A small, round fruit with a bright red skin and a sweet, juicy flesh.")
trie.insert("date", "A sweet, brown fruit from a palm tree.")

#neighbors = trie.search_knn("appl", 3)

#print("Nearest neighbors:")
#for i, neighbor in enumerate(neighbors):
    #print(f"{i + 1}. {neighbor}")
